In [1]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('..', 'TFIDF'))
sys.path.append(os.path.join('..', 'util'))
from timer import Timer

In [2]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:20000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_md.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_md.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000


In [ ]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
# tfidf.calculate_tfidf_norms()
# tfidf.calculate_cossim(question_texts[0])
serial_simhashes = tfidf.calculate_simhashes()
serial_distances = tfidf.calculate_distances()

In [14]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 64)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances = np.asarray(tfidf_c.calculate_distances())

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 5.96046447754e-06
Time for load_questions: 0.384518146515
Time for load_indices: 0.123473167419
Time for init_tfs: 3.90998792648
Time for create_tfs: 0.801409959793
Time for calculate_idf: 0.444927930832
Time for int_tfidfs: 4.74212384224
Time for calculate_tfidfs: 0.927010059357
Time for calculate_simhashes: 0.741272926331
Time for calculate distances: 5.96597194672


In [5]:
# Verification
print np.linalg.norm(cython_tfs - serial_tfs)
print np.linalg.norm(cython_idf - serial_idf)
print np.linalg.norm(cython_tfidfs - serial_tfidfs)
print np.linalg.norm(cython_simhashes - serial_simhashes)
print np.linalg.norm(cython_distances - serial_distances)

NameError: name 'serial_tfs' is not defined

In [6]:
#The following are from https://yesteapea.wordpress.com/2013/03/03/counting-the-number-of-set-bits-in-an-integer/
def numBits64(i):
    i = i - ((i >> np.uint64(1)) & np.uint64(0x5555555555555555))
    i = (i & np.uint64(0x3333333333333333)) + ((i >> np.uint64(2)) & np.uint64(0x3333333333333333))
    i = ((i + (i >> np.uint64(4))) & np.uint64(0x0F0F0F0F0F0F0F0F))
    return (i*(np.uint64(0x0101010101010101)))>>np.uint64(56)

def numBits32(i):
    i = i - ((i >> 1) & 0x55555555)
    i = (i & 0x33333333) + ((i >> 2) & 0x33333333)
    i = ((i + (i >> 4)) & 0x0F0F0F0F)
    return (i*(0x01010101))>>24

In [ ]:
A = np.array([serial_simhashes] * len(serial_simhashes))
B = np.copy(A)
distances = numBits64(A ^ B.T)

In [15]:
from sklearn.cluster import DBSCAN

In [16]:
db = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances)

In [17]:
sum(map(lambda x: 1 if x >= 0 else 0, db.labels_))

952

In [11]:
set(db.labels_)

In [12]:
all_one_labels = []
counter = 0
for i in xrange(len(db.labels_)):
    if db.labels_[i] == -1:
        counter += 1
    elif db.labels_[i] == 4:
        all_one_labels.append(i)
print counter
print all_one_labels

19048
[144, 4693, 7561, 9166, 12819, 17820, 17919]


In [13]:
for i in all_one_labels: 
    print " ".join(question_texts[i])
    print ""

i am quit new to work with jar and from read onlin i think what i am describ is possibl but i am not sure about it . if anyon ha an altern pipelin , i welcom that as well . thank for your time . i would like to do the follow : i have a java program run on my machin and i would like to accept request jar dure runtim from a remot user . the user submit the jar and i would like to add the class in the jar ( which implement some predefin interfac ) to my classpath and call function that are implement in the submit jar . i am use thrift as my commun platform .

i have 2 gb of disk space and i am upload 2 jar file that both add up to a littl less than 360 mb . but whenev i upload these jar file with a coupl other file ( that are not even big ) and whenev i push my code to upstream after edit it a bit , my disk quota reach 100 % . whi is thi the case ? how is the server automat fill up each time i edit the code and especi after i add my jar file ? i am use maven to build the jar file and am a

Now doing it for the 32BIT cython distances

In [ ]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 32)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances32 = tfidf_c.calculate_distances()

In [ ]:
db32 = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances32)

In [ ]:
sum(map(lambda x: 1 if x >= 0 else 0, db.labels_))